In [251]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from collections import Counter

In [154]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
16,tt2446042,48000000,325771424,Taken 3,Liam Neeson|Forest Whitaker|Maggie Grace|Famke...,Olivier Megaton,It Ends Here,Ex-government operative Bryan Mills finds his ...,109,Crime|Action|Thriller,Twentieth Century Fox Film Corporation|M6 Film...,1/1/2015,6.1,2015
1391,tt0418819,15000000,46770602,Land of the Dead,Simon Baker|John Leguizamo|Dennis Hopper|Rober...,George A. Romero,The dead shall inherit the Earth.,The world is full of zombies and the survivors...,93,Horror,Universal Pictures|Atmosphere Entertainment MM...,6/24/2005,5.8,2005
1763,tt0427309,15000000,30226144,The Great Debaters,Denzel Washington|Nate Parker|Forest Whitaker|...,Denzel Washington,"When the nation was in need, he inspired them ...",The true story of a brilliant but politically ...,126,Drama,The Weinstein Company|Harpo Films|Marshall Pro...,12/27/2007,6.6,2007
1634,tt0364751,19000000,69631118,Without a Paddle,Seth Green|Matthew Lillard|Dax Shepard|Antony ...,Steven Brill,"The call of the wild, the thrill of adventure....","Three friends, whose lives have been drifting ...",95,Action|Adventure|Comedy|Thriller,Paramount Pictures|De Line Pictures,8/20/2004,5.2,2004
366,tt1263670,7000000,39462438,Crazy Heart,Jeff Bridges|Maggie Gyllenhaal|Colin Farrell|R...,Scott Cooper,"The harder the life, the sweeter the song.",When reporter Jean Craddock interviews Bad Bla...,112,Drama|Music|Romance,Fox Searchlight Pictures|Butcher's Run Films|I...,12/16/2009,6.4,2009


In [155]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [156]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...
data['profit'] = data['revenue'] - data['budget']
data['genres'] = data['genres'].str.split('|')
data['cast'] = data['cast'].str.split('|')
data['production_companies'] = data['production_companies'].str.split('|')
data['director'] = data['director'].str.split('|')

def extract_month(date):
    return str(date).split('/')[0]  
data['release_month'] = data.release_date.apply(extract_month)
data['title_length'] = data['original_title'].apply(lambda t: len(t))
data['words_count'] = data['overview'].apply(lambda t: len(t.split(' ')))

In [157]:
data.head(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,title_length,words_count
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]","[Universal Studios, Amblin Entertainment, Lege...",6/9/2015,6.5,2015,1363528810,6,14,26
1,tt1392190,150000000,378436354,Mad Max: Fury Road,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",[George Miller],What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,"[Action, Adventure, Science Fiction, Thriller]","[Village Roadshow Pictures, Kennedy Miller Pro...",5/13/2015,7.1,2015,228436354,5,18,110
2,tt2908446,110000000,295238201,Insurgent,"[Shailene Woodley, Theo James, Kate Winslet, A...",[Robert Schwentke],One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,"[Adventure, Science Fiction, Thriller]","[Summit Entertainment, Mandeville Films, Red W...",3/18/2015,6.3,2015,185238201,3,9,22
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...",[J.J. Abrams],Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,"[Action, Adventure, Science Fiction, Fantasy]","[Lucasfilm, Truenorth Productions, Bad Robot]",12/15/2015,7.5,2015,1868178225,12,28,26
4,tt2820852,190000000,1506249360,Furious 7,"[Vin Diesel, Paul Walker, Jason Statham, Miche...",[James Wan],Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,"[Action, Crime, Thriller]","[Universal Pictures, Original Film, Media Righ...",4/1/2015,7.3,2015,1316249360,4,9,14


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [158]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [159]:
data[data['budget'] == data['budget'].max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 2

In [160]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [161]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals(tt0279111)'

In [162]:
data[data['runtime'] == data['runtime'].max()][['original_title', 'imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [163]:
data[data['runtime'] == data['runtime'].min()][['original_title', 'imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


In [164]:
answers['3'] = 'Winnie the Pooh(tt1449283)'

# 4. Какова средняя длительность фильмов?


In [165]:
data['runtime'].mean()

109.6585494970884

In [166]:
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [167]:
data['runtime'].median()

107.0

In [168]:
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [169]:
data[data['profit'] == data['profit'].max()][['original_title', 'imdb_id', 'profit']]

,original_title,imdb_id,profit
239,Avatar,tt0499549,2544505847


In [170]:
answers['6'] = 'Avatar(tt0499549)'

# 7. Какой фильм самый убыточный? 

In [171]:
data[data['profit'] == data['profit'].min()][['original_title', 'imdb_id', 'profit']]

,original_title,imdb_id,profit
1245,The Lone Ranger,tt1210819,-165710090


In [172]:
answers['7'] = 'The Lone Ranger(tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [173]:
len(data[data['revenue'] > data['budget']])

1478

In [174]:
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [175]:
sample2008 = data[data['release_year'] == 2008]
sample2008[sample2008['profit'] == sample2008['profit'].max()][['original_title', 'imdb_id', 'profit']]

,original_title,imdb_id,profit
599,The Dark Knight,tt0468569,816921825


In [176]:
sample2008[sample2008['revenue'] == sample2008['revenue'].max()][['original_title', 'imdb_id', 'profit']]

,original_title,imdb_id,profit
599,The Dark Knight,tt0468569,816921825


In [177]:
answers['9'] = 'The Dark Knight(tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [178]:
sample1214 = data.query('release_year >= 2012 & release_year <= 2014')
sample1214[sample1214['profit'] == sample1214['profit'].min()][['original_title', 'imdb_id', 'profit']]

,original_title,imdb_id,profit
1245,The Lone Ranger,tt1210819,-165710090


In [179]:
answers['10'] = 'The Lone Ranger(tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [180]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
df = data.explode('genres')
df.genres.value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: genres, dtype: int64

In [181]:
answers['11'] = 'Drama'

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [182]:
df.query('profit > 0').genres.value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: genres, dtype: int64

In [183]:
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [184]:
df = data.explode('director')
df.groupby(['director'])['revenue'].sum().sort_values(ascending=False)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 997, dtype: int64

In [185]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [186]:
df = data.explode('genres')
#ищем все фильмы в стиле Action
df = df.query('genres == "Action"')
#группируем их по режисеру
df.explode('director').director.value_counts()

Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Antoine Fuqua         6
Ridley Scott          6
                     ..
Philip Kaufman        1
David Mamet           1
Rob Pritts            1
Don Bluth             1
Jason Eisener         1
Name: director, Length: 364, dtype: int64

In [187]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [188]:
df = data.query('release_year == 2012')
df.explode('cast').groupby('cast')['revenue'].sum().sort_values(ascending=False)

cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Chris Evans          1519557910
Robert Downey Jr.    1519557910
                        ...    
Jason Bateman           3428048
Danny Huston            2106557
Sami Gayle              2106557
Josh Lucas              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

In [189]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [190]:
df = data[data['budget'] > data['budget'].mean()]
df.explode('cast').groupby('cast')['budget'].count().sort_values(ascending=False)

cast
Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Eddie Murphy         15
Samuel L. Jackson    15
                     ..
Leslie Bibb           1
Leonard Nimoy         1
Lennie James          1
Lena Olin             1
50 Cent               1
Name: budget, Length: 1505, dtype: int64

In [191]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [192]:
df = data.explode('cast')
df.query('cast == "Nicolas Cage"').explode('genres').genres.value_counts()
#df.explode('cast').groupby('cast')['budget'].count().sort_values(ascending=False)

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Mystery             3
Family              3
Animation           3
History             2
Horror              1
War                 1
Romance             1
Name: genres, dtype: int64

In [193]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [194]:
df = data.explode('production_companies')
df = df[df.production_companies.str.match('Paramount Pictures')]
df[df.profit == df.profit.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
925,K-19: The Widowmaker,tt0267626


In [195]:
answers['18'] = 'K-19: The Widowmaker(tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [196]:
data.groupby('release_year')['revenue'].sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

In [197]:
answers['19'] = 2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [198]:
df = data.explode('production_companies')
df = df[df.production_companies.str.contains('Warner Bros')]
df.groupby('release_year')['profit'].sum().sort_values(ascending=False)

release_year
2014    2292949646
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1844008221
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1282142579
2012    1258020056
2002    1022709901
2015     870368348
2006     625351872
2000     452631386
Name: profit, dtype: int64

In [199]:
answers['20'] = 2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [200]:
data.groupby('release_month')['imdb_id'].count().sort_values(ascending=False)

release_month
9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: imdb_id, dtype: int64

In [201]:
answers['21'] = 9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [202]:
len(data[data['release_month'].isin(['6', '7', '8'])])

450

In [203]:
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [204]:
df = data[data['release_month'].isin(['12', '1', '2'])]
df.explode('director').director.value_counts()

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
Martin Scorsese      4
Nancy Meyers         4
                    ..
Gavin O'Connor       1
Ronald F. Maxwell    1
Michel Gondry        1
Mark Dindal          1
Renny Harlin         1
Name: director, Length: 358, dtype: int64

In [205]:
df = data.explode('director')
df = df.query('release_month in ["12", "1", "2"]')
dfpt = df.pivot_table(index='director', columns='release_month', values='original_title', aggfunc='count', fill_value=0)
num = dfpt.sum(axis=1).max()
name = dfpt.sum(axis=1).idxmax()
print(f'{name} - {num}')

Peter Jackson - 7


In [212]:
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [225]:
data.explode('production_companies').groupby('production_companies')['title_length'].max().sort_values(ascending=False).index[0]

'Four By Two Productions'

In [213]:
answers['24'] = 'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [224]:
data.explode('production_companies').groupby('production_companies')['words_count'].mean().sort_values(ascending=False).index[0]

'Midnight Picture Show'

In [214]:
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [249]:
df = data[data['vote_average'] > data['vote_average'].quantile(q=0.99)]
df[['original_title', 'vote_average']].sort_values(by='vote_average', ascending=False)

,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
125,The Imitation Game,8.0
119,Guardians of the Galaxy,7.9
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
872,The Pianist,7.9


In [250]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [338]:
cnt = Counter()
for i in range(len(data)):
    pairs = itertools.combinations(data.cast[i], 2)
    for pair in pairs:
        if (pair not in cnt):
            cnt[pair] = 0
        cnt[pair] += 1
cnt.most_common(1)


[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

In [339]:
answers['27'] = 'Daniel Radcliffe, Rupert Grint'

ВАРИАНТ 2

# Submission

In [340]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals(tt0279111)',
 '3': 'Winnie the Pooh(tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar(tt0499549)',
 '7': 'The Lone Ranger(tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight(tt0468569)',
 '10': 'The Lone Ranger(tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker(tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe, Rupert Grint'}

In [341]:
# и убедиться что ни чего не пропустил)
len(answers)

27